# NOT WORKING ON GOOGLE COLAB, USE JUPYTER LAB/NOTEBOOK

## INSTALL PYMOL WITH THE FOLLOWING COMMAND IN ANACONDA
### conda install -c conda-forge pymol-open-source

In [1]:
import os

In [6]:
xbb1_mutations = ['T19I',
 'L24S',
 'DEL25/27',
 'V83A',
 'G142D',
 'DEL144/144',
 'H146Q',
 'Q183E',
 'V213E',
 'G252V',
 'G339H',
 'R346T',
 'L368I',
 'S371F',
 'S373P',
 'S375F',
 'T376A',
 'D405N',
 'R408S',
 'K417N',
 'N440K',
 'V445P',
 'G446S',
 'N460K',
 'S477N',
 'T478K',
 'E484A',
 'F490S',
 'Q498R',
 'N501Y',
 'Y505H',
 'D614G',
 'H655Y',
 'N679K',
 'P681H',
 'N764K',
 'D796Y',
 'Q954H',
 'N969K'
]
variants_mutations = {"XBB.1.9.1": ['F486P'], "XBB2.3": ['V252G', 'D253G', 'F486P', 'P521S'], "XBB.1.5": ['F486P'], "XBB.1.16": ['E180V', 'T478R', 'F486P'], "EG.5.1": ['Q52H', 'F456L', 'F486P'],}
cwd = os.getcwd()
folders = {"RBD_ACE2": cwd+os.sep+"PDBs"+os.sep+"RBD_ACE2"}
xbb_variant_pdb = {"RBD_ACE2": cwd+os.sep+"PDBs"+os.sep+"RBD_ACE2"+os.sep+"xbb.1_rbd_ace2.pdb"}
ace2_chain = "A"

In [3]:
import numpy as np

In [4]:
import pymol
from pymol import cmd
from pymol import stored
from pymol import selector

In [5]:
def readPDBFile(pdbFilePath):

    atoms = []
    with open(pdbFilePath) as pdbfile:
        for line in pdbfile:
            if line[:4] == 'ATOM':
              # Split the line
                splitted_line = [line[:6], line[6:11], line[12:16], line[17:20], line[21], line[22:26], line[30:38], line[38:46], line[46:54], line[56:61], line[62:66]]
                atoms.append(splitted_line)
                # To format again the pdb file with the fields extracted
                #print("%-6s%5s %4s %3s %s %4s    %8s%8s%8s   %3s%3s\n"%tuple(splitted_line))
    return np.array(atoms)


def getModelledResidues(pdbFilePath):

    modelled_residues = []
    atoms = readPDBFile(pdbFilePath)

    last_resi = 0
    for i, atom in enumerate(atoms):

        resn = atom[3] #res name
        resi = int(atom[5]) #res index
        chain = atom[4]

        if resi != last_resi:
            if resi not in modelled_residues:
                modelled_residues.append(resi)
                last_resi = resi

    return modelled_residues

all_res_names = {
    "A":"ALA",
    "C":"CYS",
    "D":"ASP",
    "E":"GLU",
    "F":"PHE",
    "G":"GLY",
    "H":"HIS",
    "I":"ILE",
    "K":"LYS",
    "L":"LEU",
    "M":"MET",
    "N":"ASN",
    "P":"PRO",
    "Q":"GLN",
    "R":"ARG",
    "S":"SER",
    "T":"THR",
    "V":"VAL"
}


In [8]:
for variant, mutations in variants_mutations.items():
    print(variant, mutations)
    for conformation, folder in folders.items():
        print(conformation)
        xbb_variant_conf = xbb_variant_pdb[conformation]
        all_resi_spike = getModelledResidues(xbb_variant_conf)

        spike_chains = ["B"]
        cmd.load(xbb_variant_conf)

        for mutation in mutations:

            print(mutation)
            from_resn_1 = mutation[0]
            resi = mutation[1:len(mutation)-1]
            to_resn_1 = mutation[len(mutation)-1]
            to_resn_3 = all_res_names[to_resn_1]
            print()

            if int(resi) in all_resi_spike:
                print("Mutating resi {} from {} to {}...".format(resi, from_resn_1, to_resn_1))
                for chain in spike_chains:
                    cmd.wizard("mutagenesis")
                    cmd.refresh_wizard()

                    # Mutate
                    cmd.get_wizard().set_mode("{}".format(to_resn_3))
                    cmd.get_wizard().do_select("{}/{}/".format(chain, resi))
                    cmd.frame("1")
                    cmd.get_wizard().apply()
                    cmd.set_wizard("")
            else:
                print("Residue {} not modelled in XBB 1.0".format(resi))
        # Save
        cmd.save(folder+os.sep+variant+"_"+conformation+".pdb")
        cmd.delete("all")

XBB.1.9.1 ['F486P']
RBD_ACE2
F486P

Mutating resi 486 from F to P...
Selected!
 Mutagenesis: 2 rotamers loaded.
 Rotamer 1/2, strain=28.50
XBB2.3 ['V252G', 'D253G', 'F486P', 'P521S']
RBD_ACE2
V252G

Mutating resi 252 from V to G...
Selected!
D253G

Mutating resi 253 from D to G...
Selected!
F486P

Mutating resi 486 from F to P...
Selected!
 Mutagenesis: 2 rotamers loaded.
 Rotamer 1/2, strain=28.50
P521S

Mutating resi 521 from P to S...
Selected!
 Mutagenesis: 3 rotamers loaded.
 Rotamer 3/3, strain=19.44
XBB.1.5 ['F486P']
RBD_ACE2
F486P

Mutating resi 486 from F to P...
Selected!
 Mutagenesis: 2 rotamers loaded.
 Rotamer 1/2, strain=28.50
XBB.1.16 ['E180V', 'T478R', 'F486P']
RBD_ACE2
E180V

Mutating resi 180 from E to V...
Selected!
T478R

Mutating resi 478 from T to R...
Selected!
 Mutagenesis: 21 rotamers loaded.
 Rotamer 20/21, strain=17.66
F486P

Mutating resi 486 from F to P...
Selected!
 Mutagenesis: 2 rotamers loaded.
 Rotamer 1/2, strain=28.50
EG.5.1 ['Q52H', 'F456L', 'F486P'